# Visualizing Convnet Filters
### Details
- This file will experiment with the Xception convolutional base by finding the patterns for which its filters are most receptive to

In [1]:
# imports
import tensorflow as tf
from tensorflow import keras
from keras import layers
import numpy as np
import matplotlib.pyplot as plt


c:\users\danie\pycharmprojects\tensorflowtest\venv\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\users\danie\pycharmprojects\tensorflowtest\venv\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\users\danie\pycharmprojects\tensorflowtest\venv\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
c:\users\danie\pycharmprojects\tensorflowtest\venv\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# load in model
model = keras.applications.xception.Xception(weights="imagenet", include_top=False)

# find the list of significant layers (Conv2D and SeparableConv2D layers)
for layer in model.layers:
    if isinstance(layer, (layers.Conv2D, layers.SeparableConv2D)):
        print(layer.name)


block1_conv1
block1_conv2
block2_sepconv1
block2_sepconv2
conv2d
block3_sepconv1
block3_sepconv2
conv2d_1
block4_sepconv1
block4_sepconv2
conv2d_2
block5_sepconv1
block5_sepconv2
block5_sepconv3
block6_sepconv1
block6_sepconv2
block6_sepconv3
block7_sepconv1
block7_sepconv2
block7_sepconv3
block8_sepconv1
block8_sepconv2
block8_sepconv3
block9_sepconv1
block9_sepconv2
block9_sepconv3
block10_sepconv1
block10_sepconv2
block10_sepconv3
block11_sepconv1
block11_sepconv2
block11_sepconv3
block12_sepconv1
block12_sepconv2
block12_sepconv3
block13_sepconv1
block13_sepconv2
conv2d_3
block14_sepconv1
block14_sepconv2


In [8]:
# create a feature extraction model
layer_name = "block6_sepconv3"
layer = model.get_layer(name=layer_name)
feature_extractor = keras.Model(inputs=model.input, outputs=layer.output)

# a function that computes the loss
def compute_loss(image, filter_index):
    activation = feature_extractor(image)
    filter_activation = activation[0,2:-2, 2:-2,filter_index]
    return tf.reduce_mean(filter_activation)

# perform one step of gradient ascent - note that normalizing the gradients ensures that the updates are a smaller range
@tf.function
def gradient_ascent_step(image, filter_index, learning_rate):
    with tf.GradientTape() as tape:
        loss = compute_loss(image, filter_index)
    grads = tape.gradient(loss, image)
    grads = tf.math.l2_normalize(grads)
    image.assign_add(learning_rate * grads)
    return image

# generates a filter pattern by continually running gradient ascent on on the filter
img_width, img_height = 200, 200
def generate_filter_pattern(filter_index):
    iterations = 30
    learning_rate = 10.
    image = tf.random.uniform(minval=0, maxval=0, shape=(1, img_width, img_height, 3))
    for i in range(iterations):
        image = gradient_ascent_step(tf.Variable(image), filter_index, learning_rate)
    return image[0].numpy()

# makes an image fit for viewing
def deprocess_image(image: np.ndarray):
    image = (image - image.mean()) / image.std()
    image = image * 64 + 128
    image = np.clip(image, 0, 255).astype("uint8")
    image = image[25:-25, 25:-25, :]
    return image

all_images = []
for filter_index in range(min(64, layer.output_shape[-1])):
    print(f"Processing Filter {filter_index}")
    image = deprocess_image(generate_filter_pattern(filter_index=filter_index))
    all_images.append(image)

margin = 5
n = 8
cropped_width = img_width - 25 * 2
cropped_height = img_height - 25 * 2
width = n * cropped_width + (n-1) * margin
height = n * cropped_height + (n-1) * margin
stitched_filters = np.zeros((width, height, 3))

for i in range(n):
    for j in range(n):
        if i * n + j >= len(all_images):
            break
        image = all_images[i * n + j]
        stitched_filters[
            (cropped_width+margin) * i:(cropped_width+margin)*i + cropped_width,
            (cropped_height+margin)*j:(cropped_height+margin)*j+cropped_height, :
        ] = image
keras.utils.save_img(f"filters_for_layer_{layer_name}.png", stitched_filters)


Processing Filter 0
Processing Filter 1
Processing Filter 2
Processing Filter 3
Processing Filter 4
Processing Filter 5
Processing Filter 6
Processing Filter 7
Processing Filter 8
Processing Filter 9
Processing Filter 10
Processing Filter 11
Processing Filter 12
Processing Filter 13
Processing Filter 14
Processing Filter 15
Processing Filter 16
Processing Filter 17
Processing Filter 18
Processing Filter 19
Processing Filter 20
Processing Filter 21
Processing Filter 22
Processing Filter 23
Processing Filter 24
Processing Filter 25
Processing Filter 26
Processing Filter 27
Processing Filter 28
Processing Filter 29
Processing Filter 30
Processing Filter 31
Processing Filter 32
Processing Filter 33
Processing Filter 34
Processing Filter 35
Processing Filter 36
Processing Filter 37
Processing Filter 38
Processing Filter 39
Processing Filter 40
Processing Filter 41
Processing Filter 42
Processing Filter 43
Processing Filter 44
Processing Filter 45
Processing Filter 46
Processing Filter 47
Pr